# Evaluate the effects of PS on P, TV, F, sEAdi etc

## Read-me
<!-- TODO ... Insert description ... -->

The code to consecutively:
<!-- TODO  Update code steps-->

## 1. Import code libraries

In [ ]:
# Standard code libraries
import os
import sys
import glob
from pathlib import Path

import scipy
from scipy import interpolate as interp
from scipy.optimize import curve_fit
import pandas as pd
import numpy as np
import numpy.matlib

import matplotlib.pyplot as plt
import ipywidgets as widgets
from datetime import datetime

import neurokit2 as nk

%matplotlib widget

In [ ]:
# Custom code libraries from the ReSurfEMG repository
# It uses the ReSurfEMG library version v0.2.1

import resurfemg.preprocessing.ecg_removal as ecg_rm
import resurfemg.preprocessing.envelope as evl
import resurfemg.preprocessing.filtering as filt
import resurfemg.postprocessing.features as feat

from resurfemg.data_connector.tmsisdk_lite import Poly5Reader

from resurfemg.config.config import Config
config = Config()

In [ ]:
def moving_baseline(signal, fs, window_length):
    #  Augmented moving baseline for EMGdi signals for baseline crossing detection

    # 2.a. Calculate the "default" moving baseline over the sEAdi data over a 7.5s 
    #     window
    rolling_baseline = np.zeros((len(signal), ))
    for idx in range(0, len(signal), int(fs/5)):
        start_i = max([0, idx-int(window_length/2)])
        end_i = min([len(signal), idx+int(window_length/2)])
        baseline_value = np.nanpercentile(signal[start_i:end_i], 33)
        
        for i in range(idx, min([idx+int(fs/5), len(signal)])):
            rolling_baseline[i] = baseline_value
    
    return rolling_baseline

def augmented_moving_baseline(signal, fs, window_length, augmented_perc):
    # 2.a. Calculate the "default" moving baseline over the sEAdi data over 
    #       a 7.5s window
    default_rolling_baseline = moving_baseline(signal, fs, window_length)

    # 2.b. Rolling standard deviation and mean over 7.5s window
    baseline_series = pd.Series(default_rolling_baseline)
    baseline_std = baseline_series.rolling(window_length, 
                                    min_periods=1, 
                                    center=True).std().values
    baseline_mean = baseline_series.rolling(window_length, 
                                    min_periods=1, 
                                    center=True).mean().values

    # 2.c. Augmented signal: EMG + abs([dEMG/dt]_smoothed)
    ma_window = fs//2
    # augmented_perc = 25
    perc_window = fs

    s = pd.Series(signal - default_rolling_baseline)
    s_MA = s.rolling(window=ma_window, center=True).mean().values
    ds_dt = (s_MA[1:] - s_MA[:-1] ) * fs
    s_aug = signal[:-1] + np.abs(ds_dt)

    # 2.d. Run the moving median filter over the augmented signal to obtain the 
    #       baseline
    s_aug_rolling_baseline = np.zeros(
        (len(signal)-1, ))

    for idx in range(0, int(len(signal)-1), perc_window):
        start_i = max([0, idx-int(window_length)])
        end_i = min([len(signal)-1, idx+int(window_length)])

        baseline_value = np.nanpercentile(
            s_aug[start_i:end_i], augmented_perc)
        
        for i in range(idx, min([idx+int(perc_window), len(signal)-1])):
            s_aug_rolling_baseline[i] = 1.2 * baseline_value
    
    return s_aug_rolling_baseline, baseline_std, baseline_mean

## Initiation of output folder for experiments

In [ ]:
# Input data path - The main directory where all data is loaded from
root_patient_data_directory = \
    config.get_directory('root_patient_data_directory')

# Output data - General path to dir for saving .csvs and plots
main_output_dir = os.path.join(config.get_directory('preprocessed'),
                    '2024-03_PS_exploration_QRS_detection_baseline_removal')

if not os.path.exists(main_output_dir):
    os.makedirs(main_output_dir)

patient_idx = 0

## 2. Load the ventilator and sEMG data

2.a Select a patient

In [ ]:
# Select the patient of interest
# Expected data structure:
# - Patient_01
# -- Measurement_date_XXXX_XX_01
# --- 001_Baseline
# --- 002_PS_step_01
# --- 003_PS_step_02
# --- 004_PS_step_03
# --- 005_PS_step_04
# -- Measurement_date_XXXX_XX_03
# --- 001_Baseline
# --- 002_PS_step_01
# --- 003_PS_step_02
# --- 004_PS_step_03
# --- 005_PS_step_04
# -- Patient_02
# -- Measurement_date_XXXX_XX_01
# etc.

# NB Run this cell once per patient!

patient_folders = glob.glob(
        os.path.join(root_patient_data_directory, '**',''), 
        recursive=False)

patients = []
for folder in patient_folders:
    name = Path(folder).parts[-1]
    patients.append(name)

patients.sort()

btn_pt = widgets.Dropdown(  
    options=patients,
    value=patients[patient_idx],
    description='Select patient:',
    disabled=False,
)

date_idx = 0

display(btn_pt)

2.b Select a measurement date, or PS trial

In [ ]:
# Select the PS trial of interest for the selected patient
# measurement_date ~ PEEP-trial
 
# NB Run this cell once per patient/PEEP trial combination

patient = btn_pt.value
patient_idx =btn_pt.index

measurement_folders = glob.glob(
    os.path.join(root_patient_data_directory, patient, '**',''),
    recursive=False)
measurement_dates = []

for folder in measurement_folders:
    name = Path(folder).parts[-1]
    measurement_dates.append(name)

measurement_dates.sort()

# Initialise the analysis: empty the output parameter list and start at the 
# baseline measurement (index 0)
di_data_list = []
para_data_list = []
Paw_data_list = []
Vvent_data_list =[]
ecg_data_list = []

ecg_data_all_dict = dict()
df_ecg_all_dict = dict()
channel_list = ['ecg', 'di', 'para']
columns_ecg_all = ['patient', 'measurement',
              'ecg_minmax', 'ecg_min', 'ecg_max', 'ecg_method']
for idx, channel  in enumerate(channel_list):
    ecg_data_all_dict[channel] = []
    df_ecg_all_dict[channel] = pd.DataFrame(ecg_data_all_dict[channel], columns=columns_ecg_all)

PS_step_idx = 0
plt.close('all')

btn_measurement = widgets.Dropdown(
    options=measurement_dates,
    value=measurement_dates[date_idx],
    description='Select measurement date:',
    parasabled=False,
)
display(btn_measurement)

In [ ]:
# Set the default pipeline parameters
# Gating settings
gate_width_default = 0.10
gate_threshold_default = 0.30
gate_ECG_shift_default = -10
gate_twice = False

# RMS window
RMS_window_ms_default = 200

# Peak detection settings
time_shift_default = 0.5 - RMS_window_ms_default/1000/2
sEAdi_prominence_factor_default = 0.5
sEApara_prominence_factor_default = 0.5

2.c Select a PS step

In [ ]:
# Select the PS step of interest for the selected patient/measurement_date

# NB Re-run this cell for each new PEEP trial, as it also empties output 
# parameter list (di_data_list)!


# Create output data folders
measurement_date = btn_measurement.value
date_idx = btn_measurement.index

output_path = os.path.join(main_output_dir, patient, measurement_date)
if not os.path.exists(output_path):
    os.makedirs(output_path)

# Identify all PS step folders:
root_emg_directory = os.path.join(
    root_patient_data_directory, patient, measurement_date)

emg_pattern = os.path.join(root_emg_directory, '**/*.Poly5')
emg_and_vent_files = glob.glob(emg_pattern, recursive=True)

emg_files = []
vent_files = []
plt.close('all')

for file in emg_and_vent_files:
    if 'Draeger' in file:
        vent_files.append(file)
    else:
        emg_files.append(file)

emg_files.sort()
vent_files.sort()

list_of_numbers_string = []

for i in range(len(emg_files)):
    list_of_numbers_string.append(Path(emg_files[i]).parts[-2])

# Select the PEEP step of interest. The selection menu initialises at the third 
# but last (index -4) recording. The PEEP steps are named after the folders 
# containing the data files (.poly5) of interest.

btn_PS_step = widgets.Dropdown(
    options=list_of_numbers_string,
    value= list_of_numbers_string[PS_step_idx],
    description='Picked File:',
    disabled=False,
)
display(btn_PS_step)

In [ ]:
# Process selected option: the PEEP step of interest  
PS_step_chosen = btn_PS_step.value
PS_step_idx = int(btn_PS_step.index)
emg_file_chosen = emg_files[PS_step_idx]
vent_file_chosen = vent_files[PS_step_idx]
print("The chosen files are:\n", emg_file_chosen, '\n', vent_file_chosen)

In [ ]:
# Load the EMG and ventilator data recordings from the selected folders.
data_emg = Poly5Reader(emg_file_chosen)
data_vent = Poly5Reader(vent_file_chosen)
data_emg_samples = data_emg.samples[:data_emg.num_samples]
fs_emg = data_emg.sample_rate
data_vent_samples = data_vent.samples[:data_vent.num_samples]
fs_vent = data_vent.sample_rate

# Define the time series of the EMG and ventilator recordings
y_emg = data_emg_samples
# y_emg = data_emg_samples[:, :350*2048]
# Reshufle the channels if necessary
# y_emg = np.array([-data_emg_samples[0, :], 
#                    data_emg_samples[1, :], 
#                    data_emg_samples[2, :]])
# # Reshufle the channels if necessary
# y_emg = np.array([-data_emg_samples[2, :], 
#                    data_emg_samples[0, :], 
#                    -data_emg_samples[1, :]])
y_vent = data_vent_samples

# Define the time axes
t_emg = np.array([i/fs_emg for i in range(len(y_emg[0, :]))])
t_vent = np.array([i/fs_vent for i in range(len(y_vent[0, :]))])

# Default settings for window of interest
# manoeuvres (Pocc)
# t_start_default = t_vent[-1]-61
# t_end_default = t_vent[-1]-1
t_start_default = t_vent[0]
t_end_default = t_vent[-1]

del data_emg_samples, data_vent_samples, data_emg, data_vent

btn_plt_raw = widgets.Dropdown(
    options=['Yes', 'No'],
    value='No',
    description='Plot raw data?',
    parasabled=False,
)
display(btn_plt_raw)

In [ ]:
# Plot the raw data if wanted
if btn_plt_raw.value == 'Yes':
 
    fig, axis = plt.subplots(nrows=3, ncols=2, figsize=(10, 6), sharex=True)

    axis[0, 0].grid(True)
    axis[0, 0].plot(t_emg, y_emg[0])
    axis[0, 0].set(title='sEMG leads')
    axis[0, 0].set_ylabel('ECG (uV)')

    axis[1, 0].grid(True)
    axis[1, 0].plot(t_emg, y_emg[1])
    axis[1, 0].set_ylabel('sEMGdi (uV)')
    axis[1, 0].set_xlabel('t (s)')

    axis[2, 0].grid(True)
    axis[2, 0].plot(t_emg, y_emg[2])
    axis[2, 0].set_ylabel('sEMGpara (uV)')
    axis[2, 0].set_xlabel('t (s)')

    axis[0, 1].set(title='Ventilator data')
    axis[0, 1].grid(True)
    axis[0, 1].plot(t_vent, y_vent[0])
    axis[0, 1].set_ylabel('Paw (cmH2O)')

    axis[1, 1].grid(True)
    axis[1, 1].plot(t_vent, y_vent[1])
    axis[1, 1].set_ylabel('F (L/min)')

    axis[2, 1].grid(True)
    axis[2, 1].plot(t_vent, y_vent[2])
    axis[2, 1].set_ylabel('V (mL)')
    axis[2, 1].set_xlabel('t (s)')

## 3. Select the time window of interest

In [ ]:
start = t_start_default
end = t_end_default

start_s = int(float(start)* fs_emg)
end_s = min([int(float(end)*fs_emg), len(y_emg[0,:])-1])
start_vent_s = int(float(start)* fs_vent)
end_vent_s = min(
    [int(float(end)* fs_vent), len(y_vent[0,:])-1]
)

fig_w = (int(end_vent_s)-int(start_vent_s))//(fs_vent*80)*12

## 4.a. ECG properties extraction

In [ ]:
# Filter raw EMG signals according to AHA 1990 guidelines
y_emg_ECG_filt_AHA = filt.emg_bandpass_butter_sample(y_emg, 0.05, 150, fs_emg)

y_emg_ECG_filt = filt.emg_bandpass_butter_sample(y_emg, 1, 80, fs_emg)
# y_emg_ECG_filt = y_emg

# # See https://ieeexplore.ieee.org/document/10290007 for performance
# ecg_method = 'Zong'      # 'neurokit', 'hamilton2002', 'pantompkins1985', 'engzeemod2012'

ecg_dicts = dict()
channel_list = ['ecg', 'di', 'para']
for idx, channel in enumerate(channel_list):
    ecg_dicts[channel] = dict()

# RMS and peak detection settings
RMS_window_ecg_ms = 50
RMS_windows_samp = int(RMS_window_ecg_ms / 1000 *  fs_emg)
distance = 2048//4

# Moving baseline parameters
w_moving_baseline = 5 * fs_emg

# ecg_type = 'raw'    # 'clean'
channel = 'ecg'
channel_list = ['ecg', 'di', 'para']

T_ecg_bin_w = 0.1

for idx, channel in enumerate(channel_list):
    y_emg_ECG_filt[idx, :]
    ecg_dicts[channel]['raw'] = y_emg[idx, :]
    ecg_dicts[channel]['filt'] = y_emg_ECG_filt[idx, :]
    ecg_dicts[channel]['filt_AHA'] = y_emg_ECG_filt_AHA[idx, :]
    
    pre_samp = int(np.ceil(RMS_windows_samp/2))
    post_samp = int(np.floor(RMS_windows_samp/2))
    padding_pre = np.zeros((pre_samp,))
    padding_end = np.zeros((post_samp,))
    ecg_dicts[channel]['filt_padded'] = np.concatenate(
         (padding_pre, ecg_dicts[channel]['filt'], padding_end))

    # sEAdi_padded = evl.full_rolling_rms(emg_di_gated_padded, RMS_windows_samp)
    # sEAdi = sEAdi_padded[:-RMS_windows_samp]

    ecg_dicts[channel]['rms'] = evl.full_rolling_rms(
        ecg_dicts[channel]['filt_padded'], 
        RMS_windows_samp)
    
    ecg_dicts[channel]['rms'] = ecg_dicts[channel]['rms'][:-RMS_windows_samp]
    
    prominence = 0.5*(np.percentile(ecg_dicts[channel]['rms'], 95) 
                  - min(ecg_dicts[channel]['rms']))
    
    peaks, _ = scipy.signal.find_peaks(ecg_dicts[channel]['rms'], 
                                       prominence=prominence,
                                       distance=distance)
    ecg_dicts[channel]['peaks'] = np.array(peaks)
    ecg_dicts[channel]['method'] = 'custom'


    bin_w = 0.05

    T_peaks = (ecg_dicts[channel]['peaks'][1:]-ecg_dicts[channel]['peaks'][:-1])/fs_emg
    hist, bin_edges = np.histogram(T_peaks, bins=[x*bin_w for x in 
                                                range(int(max(T_peaks)/bin_w))])
    bin_edges = bin_edges[1:]
    pks, _ = scipy.signal.find_peaks(hist, height=max(hist)*T_ecg_bin_w)


    if len(pks) == 3:
            # print(bin_edges[pks])

            extra_pks = np.argwhere((T_peaks/bin_edges[pks[-1]] < 0.5) & (np.abs((T_peaks-bin_edges[pks[0]])/bin_edges[pks[0]]) < 0.5)) + 1
            # print(extra_pks)
    else:
        extra_pks = []   

    ecg_dicts[channel]['peaks'] = np.delete(ecg_dicts[channel]['peaks'], 
                                            extra_pks)
    
    T_peaks = (ecg_dicts[channel]['peaks'][1:]-ecg_dicts[channel]['peaks'][:-1])/fs_emg
    # hist, bin_edges = np.histogram(T_peak, bins=[x*bin_w for x in 
    #                   range(int(max(T_peak)/bin_w))])
    hist, bin_edges = np.histogram(T_peaks, bins=[x*bin_w for x in 
                      range(int(max(T_peaks)/bin_w))])
    
    bin_edges = bin_edges[1:]
    pks, _ = scipy.signal.find_peaks(hist, height=max(hist)*T_ecg_bin_w)

    if len(pks) > 1:
         print('Warning: Non-QRS complexes included in channel: ' + channel)
         print(T_peaks/bin_edges[pks[-1]])
         print(np.abs((T_peaks-bin_edges[pks[0]])/bin_edges[pks[0]]))

    ecg_dicts[channel]['mb_rms'] = moving_baseline(ecg_dicts[channel]['rms'], fs_emg, w_moving_baseline)
    ecg_dicts[channel]['mb_filt'] = moving_baseline(ecg_dicts[channel]['filt'], fs_emg, w_moving_baseline)

    # scipy.signal.argrelextrema(ecg_dicts[channel]['d_dt_mb_rms'])

len(ecg_dicts['ecg']['peaks']), len(ecg_dicts['di']['peaks']), len(ecg_dicts['para']['peaks'])

In [ ]:
# ecg_dicts[channel]['peak_upslope_idx']

In [ ]:
for idx, channel in enumerate(channel_list):
    ecg_dicts[channel]['d_dt_rms'] = (ecg_dicts[channel]['rms'][1:] - ecg_dicts[channel]['rms'][:-1])*fs_emg

    local_extremes_vec_up = scipy.signal.argrelextrema(ecg_dicts[channel]['d_dt_rms'], np.greater, order=RMS_windows_samp//2)[0]
    idxs_mat_up = (np.matlib.repmat(local_extremes_vec_up, len(ecg_dicts[channel]['peaks']), 1) 
                - np.matlib.repmat(ecg_dicts[channel]['peaks'], len(local_extremes_vec_up), 1).T)

    loc_crossings_up = np.argwhere(np.diff(np.sign(idxs_mat_up)) != 0)
    loc_crossings_up = loc_crossings_up[:, 1]

    ecg_dicts[channel]['peak_upslope_idx'] = local_extremes_vec_up[loc_crossings_up]
    ecg_dicts[channel]['peak_upslope_val'] = ecg_dicts[channel]['d_dt_rms'][ecg_dicts[channel]['peak_upslope_idx']]



    local_extremes_vec_down = scipy.signal.argrelextrema(-ecg_dicts[channel]['d_dt_rms'], np.greater, order=RMS_windows_samp//2)[0]
    idxs_mat_down = (np.matlib.repmat(local_extremes_vec_down, len(ecg_dicts[channel]['peaks']), 1) 
                - np.matlib.repmat(ecg_dicts[channel]['peaks'], len(local_extremes_vec_down), 1).T)

    loc_crossings_down = np.argwhere(np.diff(np.sign(idxs_mat_down)) != 0)
    loc_crossings_down = loc_crossings_down[:, 1] + 1

    ecg_dicts[channel]['peak_downslope_idx'] = local_extremes_vec_down[loc_crossings_down]
    ecg_dicts[channel]['peak_downslope_val'] = ecg_dicts[channel]['d_dt_rms'][ecg_dicts[channel]['peak_downslope_idx']]

In [ ]:
y_vals = ecg_dicts[channel]['rms'][ecg_dicts[channel]['peak_upslope_idx']]
dy_dt_vals = ecg_dicts[channel]['peak_upslope_val']

ecg_dicts[channel]['peak_upslope_idx_ds'] = np.array(y_vals * fs_emg // (dy_dt_vals), dtype=int)

ecg_dicts[channel]['peak_upslope_idx_ds'][
    np.abs(ecg_dicts[channel]['peak_upslope_idx_ds'])
    > (np.median(ecg_dicts[channel]['peaks'][1:]-ecg_dicts[channel]['peaks'][:-1]))] = 0
ecg_dicts[channel]['peak_upslope_start_s'] = ecg_dicts[channel]['peak_upslope_idx'] - ecg_dicts[channel]['peak_upslope_idx_ds']

channel, ecg_dicts[channel]['peaks'][:10]/fs_emg, y_vals[:10], dy_dt_vals[:10], ecg_dicts[channel]['peak_upslope_idx_ds'][:10], ecg_dicts[channel]['peak_upslope_start_s'][:10]/fs_emg

In [ ]:
# fig_hist_slopes, axis = plt.subplots(nrows=1, ncols=3, figsize=(10, 6))


# channel_list = ['ecg', 'di', 'para']

# for idx, channel  in enumerate(channel_list):
#     T_peaks = ecg_dicts[channel]['peak_downslope_val']
#     bin_w = np.max(T_peaks)/100

#     hist, bin_edges = np.histogram(T_peaks, bins=[x*bin_w for x in 
#                       range(int(max(T_peaks)/bin_w))])
#     bin_edges = bin_edges[1:]
#     pks, _ = scipy.signal.find_peaks(hist, height=max(hist)/10)

#     axis[idx].plot(bin_edges,hist)
#     axis[idx].plot(bin_edges[pks],hist[pks], 'rx')
#     axis[idx].set_title(channel)
#     axis[idx].set_xlabel('dRMS/dt (uV/s)')

# axis[0].set_ylabel('Bin count (.)')

In [ ]:
for idx, channel in enumerate(channel_list):   
    y_vals = ecg_dicts[channel]['rms'][ecg_dicts[channel]['peak_upslope_idx']]
    dy_dt_vals = ecg_dicts[channel]['peak_upslope_val']

    ecg_dicts[channel]['peak_upslope_idx_ds'] = np.array(y_vals * fs_emg // (dy_dt_vals), dtype=int).astype(np.int64)

    ecg_dicts[channel]['peak_upslope_idx_ds'][
        np.abs(ecg_dicts[channel]['peak_upslope_idx_ds'])
        > 2*(np.median(ecg_dicts[channel]['peaks'][1:]-ecg_dicts[channel]['peaks'][:-1]))] = 0
    ecg_dicts[channel]['peak_upslope_start_s'] = ecg_dicts[channel]['peak_upslope_idx'] - ecg_dicts[channel]['peak_upslope_idx_ds']

    y_vals = ecg_dicts[channel]['rms'][ecg_dicts[channel]['peak_downslope_idx']]
    dy_dt_vals = ecg_dicts[channel]['peak_downslope_val']

    ecg_dicts[channel]['peak_downslope_idx_ds'] = np.array(y_vals * fs_emg // (dy_dt_vals), dtype=int).astype(np.int64)
    ecg_dicts[channel]['peak_downslope_idx_ds'][
        np.abs(ecg_dicts[channel]['peak_downslope_idx_ds'])
        > 2*(np.median(ecg_dicts[channel]['peaks'][1:]-ecg_dicts[channel]['peaks'][:-1]))] = 0
    ecg_dicts[channel]['peak_downslope_end_s'] = ecg_dicts[channel]['peak_downslope_idx'] - ecg_dicts[channel]['peak_downslope_idx_ds']


    # while np.min(ecg_dicts[channel]['peak_upslope_start_s']) < 0:
    #     ecg_dicts[channel]['peaks'] = np.delete(ecg_dicts[channel]['peaks'], 0)
    #     ecg_dicts[channel]['peak_upslope_idx'] = np.delete(ecg_dicts[channel]['peak_upslope_idx'], 0)
    #     ecg_dicts[channel]['peak_upslope_val'] = np.delete(ecg_dicts[channel]['peak_upslope_val'], 0)
    #     ecg_dicts[channel]['peak_upslope_idx_ds'] = np.delete(ecg_dicts[channel]['peak_upslope_idx_ds'], 0)
    #     ecg_dicts[channel]['peak_upslope_start_s'] = np.delete(ecg_dicts[channel]['peak_upslope_start_s'], 0)
    #     ecg_dicts[channel]['peak_downslope_idx'] = np.delete(ecg_dicts[channel]['peak_upslope_idx'], 0)
    #     ecg_dicts[channel]['peak_downslope_val'] = np.delete(ecg_dicts[channel]['peak_upslope_val'], 0)
    #     ecg_dicts[channel]['peak_downslope_idx_ds'] = np.delete(ecg_dicts[channel]['peak_upslope_idx_ds'], 0)
    #     ecg_dicts[channel]['peak_downslope_start_s'] = np.delete(ecg_dicts[channel]['peak_upslope_start_s'], 0)

    # while np.max(ecg_dicts[channel]['peak_downslope_end_s']) >= len(ecg_dicts[channel]['rms']):
    #     ecg_dicts[channel]['peaks'] = np.delete(ecg_dicts[channel]['peaks'], -1)
    #     ecg_dicts[channel]['peak_upslope_idx'] = np.delete(ecg_dicts[channel]['peak_upslope_idx'], -1)
    #     ecg_dicts[channel]['peak_upslope_val'] = np.delete(ecg_dicts[channel]['peak_upslope_val'], -1)
    #     ecg_dicts[channel]['peak_upslope_idx_ds'] = np.delete(ecg_dicts[channel]['peak_upslope_idx_ds'], -1)
    #     ecg_dicts[channel]['peak_upslope_start_s'] = np.delete(ecg_dicts[channel]['peak_upslope_start_s'], -1)
    #     ecg_dicts[channel]['peak_downslope_idx'] = np.delete(ecg_dicts[channel]['peak_upslope_idx'], -1)
    #     ecg_dicts[channel]['peak_downslope_val'] = np.delete(ecg_dicts[channel]['peak_upslope_val'], -1)
    #     ecg_dicts[channel]['peak_downslope_idx_ds'] = np.delete(ecg_dicts[channel]['peak_upslope_idx_ds'], -1)
    #     ecg_dicts[channel]['peak_downslope_start_s'] = np.delete(ecg_dicts[channel]['peak_upslope_start_s'], -1)

    # print(y_vals[:10], dy_dt_vals[:10], fs_emg * y_vals[:10]//dy_dt_vals[:10])

bin_edges[1:], hist

In [ ]:
fig_t, axis = plt.subplots(nrows=3, ncols=2, figsize=(10, 6), sharex=True)
for idx, channel in enumerate(channel_list):
    axis[idx, 0].grid(True)
    axis[idx, 0].plot(t_emg, ecg_dicts[channel]['raw'] - np.percentile(ecg_dicts[channel]['raw'], 5))
    axis[idx, 0].plot(t_emg, ecg_dicts[channel]['filt'])
    # axis[idx, 0].plot(t_emg, ecg_dicts[channel]['mb_filt'])
    axis[idx, 0].plot(t_emg, ecg_dicts[channel]['mb_rms'])
    axis[idx, 0].plot(t_emg, ecg_dicts[channel]['rms'])
    axis[idx, 0].plot(t_emg[ecg_dicts[channel]['peak_upslope_idx']], ecg_dicts[channel]['rms'][ecg_dicts[channel]['peak_upslope_idx']], 'rx')
    axis[idx, 0].plot(t_emg[ecg_dicts[channel]['peak_downslope_idx']], ecg_dicts[channel]['rms'][ecg_dicts[channel]['peak_downslope_idx']], 'rx')

    axis[idx, 0].plot(t_emg[ecg_dicts[channel]['peak_upslope_idx']], ecg_dicts[channel]['filt'][ecg_dicts[channel]['peak_upslope_idx']], 'cx')
    axis[idx, 0].plot(t_emg[ecg_dicts[channel]['peak_downslope_idx']], ecg_dicts[channel]['filt'][ecg_dicts[channel]['peak_downslope_idx']], 'cx')

    axis[idx, 0].plot(t_emg[ecg_dicts[channel]['peak_upslope_start_s']], ecg_dicts[channel]['filt'][ecg_dicts[channel]['peak_upslope_start_s']], 'mx')
    axis[idx, 0].plot(t_emg[ecg_dicts[channel]['peak_downslope_end_s']], ecg_dicts[channel]['filt'][ecg_dicts[channel]['peak_downslope_end_s']], 'kx')

    axis[idx, 0].plot(t_emg[ecg_dicts[channel]['peaks']], 
                      ecg_dicts[channel]['rms'][ecg_dicts[channel]['peaks']], '*r')
    # if idx == 0:
    #     axis[idx, 0].plot(t_emg[ecg_dicts[channel]['peaks'][extra_pks]], 
    #                     ecg_dicts[channel]['rms'][ecg_dicts[channel]['peaks'][extra_pks]], 'xc')
    
    extra_pks
    
    axis[idx, 0].set_ylabel(channel_list[idx] + ' (uV)')
axis[idx, 0].set_xlabel('t (s)')

axis[0, 0].set(title='sEMG leads')
axis[2, 0].set_xlabel('t (s)')

axis[0, 1].set(title='Ventilator data')
axis[0, 1].grid(True)
axis[0, 1].plot(t_vent, y_vent[0])
axis[0, 1].set_ylabel('Paw (cmH2O)')

axis[1, 1].grid(True)
axis[1, 1].plot(t_vent, y_vent[1])
axis[1, 1].set_ylabel('F (L/min)')

axis[2, 1].grid(True)
axis[2, 1].plot(t_vent, y_vent[2])
axis[2, 1].set_ylabel('V (mL)')
axis[2, 1].set_xlabel('t (s)')

In [ ]:
np.max(ecg_dicts[channel]['peak_upslope_start_s'])

In [ ]:
1121616
150086

In [ ]:
fig_hist_ecg, axis = plt.subplots(nrows=1, ncols=3, figsize=(10, 6))
bin_w = 0.05

channel_list = ['ecg', 'di', 'para']

for idx, channel  in enumerate(channel_list):
    T_peaks = (ecg_dicts[channel]['peaks'][1:]
               -ecg_dicts[channel]['peaks'][:-1])/fs_emg

    hist, bin_edges = np.histogram(T_peaks, bins=[x*bin_w for x in 
                      range(int(max(T_peaks)/bin_w))])
    bin_edges = bin_edges[1:]
    pks, _ = scipy.signal.find_peaks(hist, height=max(hist)/10)

    axis[idx].plot(bin_edges,hist)
    axis[idx].plot(bin_edges[pks],hist[pks], 'rx')
    axis[idx].set_title(channel)
    axis[idx].set_xlabel('Tpeak (s)')

axis[0].set_ylabel('Bin count (.)')

In [ ]:
# Min-Max amplitude
channel_list = ['ecg', 'di', 'para']

for idx, channel  in enumerate(channel_list):
    ecg_dicts[channel]['mins'] = []
    ecg_dicts[channel]['min_idxs'] = []
    ecg_dicts[channel]['maxs'] = []
    ecg_dicts[channel]['max_idxs'] = []
    ecg_dicts[channel]['minmax'] = []

    ecg_dicts[channel]['peaks']
    win_half = int(np.median(ecg_dicts[channel]['peaks'][1:] 
                             - ecg_dicts[channel]['peaks'][:-1])//10)
    for pk_idx, R_peak  in enumerate(ecg_dicts[channel]['peaks']):
        _s_start = max([0, R_peak-win_half])
        _s_end = min([len(ecg_dicts[channel]['raw']), R_peak+win_half])
        # _signal = ecg_dicts[channel]['filt_AHA'][_s_start:_s_end]
        _signal = ecg_dicts[channel]['filt'][_s_start:_s_end]

        ecg_dicts[channel]['mins'].append(np.min(_signal))
        ecg_dicts[channel]['min_idxs'].append(
            np.argmin(_signal) + R_peak - win_half)
        ecg_dicts[channel]['maxs'].append(np.max(_signal))
        ecg_dicts[channel]['max_idxs'].append(
            np.argmax(_signal) + R_peak - win_half)
        ecg_dicts[channel]['minmax'].append(np.max(_signal) - np.min(_signal))


In [ ]:
fig_t, axis = plt.subplots(nrows=3, ncols=2, figsize=(10, 6), sharex=True)

for idx, channel in enumerate(channel_list):
    axis[idx, 0].grid(True)
    # axis[idx, 0].plot(t_emg, ecg_dicts[channel]['filt_AHA'])
    # axis[idx, 0].plot(t_emg[ecg_dicts[channel]['min_idxs']], 
    #                   ecg_dicts[channel]['filt_AHA'][ecg_dicts[channel]['min_idxs']], '*r')
    # axis[idx, 0].plot(t_emg[ecg_dicts[channel]['max_idxs']], 
    #                   ecg_dicts[channel]['filt_AHA'][ecg_dicts[channel]['max_idxs']], '*g')
    
    axis[idx, 0].plot(t_emg, ecg_dicts[channel]['filt'])
    axis[idx, 0].plot(t_emg[ecg_dicts[channel]['min_idxs']], 
                      ecg_dicts[channel]['filt'][ecg_dicts[channel]['min_idxs']], '*r')
    axis[idx, 0].plot(t_emg[ecg_dicts[channel]['max_idxs']], 
                      ecg_dicts[channel]['filt'][ecg_dicts[channel]['max_idxs']], '*g')

    axis[idx, 0].set_ylabel(channel_list[idx] + ' (uV)')

axis[idx, 0].set_xlabel('t (s)')

axis[0, 0].set(title='sEMG leads')
axis[2, 0].set_xlabel('t (s)')

axis[0, 1].set(title='Ventilator data')
axis[0, 1].grid(True)
axis[0, 1].plot(t_vent, y_vent[0])
axis[0, 1].set_ylabel('Paw (cmH2O)')

axis[1, 1].grid(True)
axis[1, 1].plot(t_vent, y_vent[1])
axis[1, 1].set_ylabel('F (L/min)')

axis[2, 1].grid(True)
axis[2, 1].plot(t_vent, y_vent[2])
axis[2, 1].set_ylabel('V (mL)')
axis[2, 1].set_xlabel('t (s)')

In [ ]:
# Plot histograms of QRS min-max amplitudes

fig_hist, axis = plt.subplots(nrows=1, ncols=3, figsize=(10, 6))
bin_w = 5

channel_list = ['ecg', 'di', 'para']

for idx, channel  in enumerate(channel_list):
    axis[idx].hist(ecg_dicts[channel]['minmax'], 
                bins=[x*bin_w for x in range(int(max(ecg_dicts[channel]['minmax'])/bin_w))])
    axis[idx].set_title(channel)
    axis[idx].set_xlabel('QRS min-max amp (uV)')
    
    axis[idx].set_xlim([np.percentile(ecg_dicts[channel]['minmax'], 1), np.percentile(ecg_dicts[channel]['minmax'], 99)])

axis[0].set_ylabel('Bin count (.)')

(np.median(ecg_dicts['ecg']['minmax']), np.median(ecg_dicts['di']['minmax']), 
 np.median(ecg_dicts['para']['minmax']))


## 11. Add the outcome data to a dataframe

In [ ]:
print(patient + ' / ' + measurement_date + ' / ' + PS_step_chosen)

In [ ]:
fig_t.savefig(main_output_dir + '/' + patient + '/' + measurement_date
              + '/' + measurement_date + '_' + patient + '_' 
              + PS_step_chosen + 'ECG_timeplots.png', 
              dpi=300)
fig_hist.savefig(main_output_dir + '/' + patient + '/' + measurement_date
              + '/' + measurement_date + '_' + patient + '_' 
              + PS_step_chosen + 'ECG_histograms.png', 
              dpi=300)


In [ ]:
# di_data_list = []
# para_data_list = []
# Paw_data_list = []
# Vvent_data_list = []
# ecg_data_list = []

# ecg_data_all_dict = dict()
# df_ecg_all_dict = dict()
# channel_list = ['ecg', 'di', 'para']
# columns_ecg_all = ['patient', 'measurement',
#               'ecg_minmax', 'ecg_min', 'ecg_max', 'ecg_method']
# for idx, channel  in enumerate(channel_list):
#     ecg_data_all_dict[channel] = []
#     df_ecg_all_dict[channel] = pd.DataFrame(ecg_data_all_dict[channel], columns=columns_ecg_all)

# # PS_step_del = '002'
# PS_step_del = PS_step_chosen
# df_di = df_di.drop(df_di[df_di['measurement'] == PS_step_del].index)
# df_para = df_para.drop(df_para[df_para['measurement'] == PS_step_del].index)
# df_Paw = df_Paw.drop(df_Paw[df_Paw['measurement'] == PS_step_del].index)
# df_Vvent = df_Vvent.drop(df_Vvent[df_Vvent['measurement'] == PS_step_del].index)
# df_ecg = df_ecg.drop(df_ecg[df_ecg['measurement'] == PS_step_del].index)

# di_data_list = list(df_di.values)W
# para_data_list = list(df_para.values)
# Paw_data_list = list(df_Paw.values)
# Vvent_data_list = list(df_Vvent.values)

# df_di

In [ ]:
# Store overall ECG parameters in a dataframe (~ table)
columns_ecg = ['patient', 'measurement',
              'ecg_ecg_minmax', 'ecg_method', 
              'ecg_di_minmax', 'di_method', 
              'ecg_para_minmax', 'para_method']

df_ecg = pd.DataFrame(ecg_data_list, columns=columns_ecg)

if len(df_ecg[df_ecg['measurement'] == PS_step_chosen]) == 0:
    data_ecg_tmp = [patient, PS_step_chosen,
                    np.median(ecg_dicts['ecg']['minmax']), 
                    ecg_dicts['ecg']['method'], 
                    np.median(ecg_dicts['di']['minmax']), 
                    ecg_dicts['di']['method'], 
                    np.median(ecg_dicts['para']['minmax']),
                    ecg_dicts['para']['method'], 
                ]
    
    ecg_data_list.append(data_ecg_tmp)
    
    df_ecg = pd.DataFrame(ecg_data_list, columns=columns_ecg)
else:
    print('Data of PS step ' + PS_step_chosen + ' already added to the ' 
          + 'dataframe. Don''t add the data twice!')
    
df_ecg

In [ ]:
# Store all ECG peaks in a in a dataframe (~ table)
columns_ecg_all = ['patient', 'measurement',
                   'ecg_min', 'ecg_max', 'ecg_minmax', 'ecg_method']

channel_list = ['ecg', 'di', 'para']
for _, channel  in enumerate(channel_list):
    if len(df_ecg_all_dict[channel][df_ecg_all_dict[channel]['measurement'] == PS_step_chosen]) == 0:
        for idx in range(len(ecg_dicts[channel]['minmax'])):
            data_ecg_all_tmp = [patient, PS_step_chosen,
                                ecg_dicts[channel]['mins'][idx],
                                ecg_dicts[channel]['maxs'][idx],
                                ecg_dicts[channel]['minmax'][idx],
                                ecg_dicts[channel]['method']]
        
            ecg_data_all_dict[channel].append(data_ecg_all_tmp)
            
        df_ecg_all_dict[channel] = pd.DataFrame(ecg_data_all_dict[channel], columns=columns_ecg_all)
    else:
        print('Data of PS step ' + PS_step_chosen + ' already added to the ' 
            + 'dataframe for channel ' +  channel + '. Don''t add the data '
            + 'twice!')

In [ ]:
# Move on to the next measurement
if PS_step_chosen in df_ecg['measurement'].to_list():
    if PS_step_idx >= 5:
        print('Data of measurement ' + PS_step_chosen + ' added to the '
              + 'dataframe. PS step ' + str(PS_step_chosen) + ' (' +
              str(len(set(df_ecg['measurement'].values))) + '/5) is '
              + 'successfully evaluated and stored. You can continue to the '
              + 'saving the overall data!')
              
    elif PS_step_chosen != list_of_numbers_string[PS_step_idx]:
        print('Data of measurement ' + PS_step_chosen + ' added to the '
              + 'dataframe. PS step ' + str(PS_step_chosen) + ' (' +
              str(len(set(df_ecg['measurement'].values))) + '/5) is '
              + 'successfully evaluated and stored. Run the next PEEP step, '
              + 'starting at step 2C.')
    else:
        PS_step_idx += 1
        if PS_step_idx >= 5:
            print('Data of measurement ' + PS_step_chosen + ' added to the '
              + 'dataframe. PS step ' + str(PS_step_chosen) + ' (' +
              str(len(set(df_ecg['measurement'].values))) + '/5) is '
              + 'successfully evaluated and stored. You can continue to the '
              + 'saving the overall data!')
        else:
            print('Data of measurement ' + PS_step_chosen + ' added to the '
              + 'dataframe. PS step ' + str(PS_step_chosen) + ' (' +
              str(len(set(df_ecg['measurement'].values))) + '/5) is '
              + 'successfully evaluated and stored. Run the next PEEP step, '
              + 'starting at step 2C.')
else:
    print('Data of measurement ' + PS_step_chosen + ' not yet added to the ' 
        + 'dataframe!')

['To next PS step'](#section_ps_step)

In [ ]:
# Store session data if complete
if ((len(set(df_ecg['measurement'].values)) < 5) |
    (measurement_date != measurement_dates[date_idx])):
    print('Warning: Not 5 PS settings evaluated yet!')
else:
    df_ecg.to_csv(main_output_dir + '/' + patient + '/' + measurement_date  
              + '/' + measurement_date + '_' + patient
              +'_ECGs.csv')
    
    for _, channel in enumerate(channel_list):
   
        df_ecg_all_dict[channel].to_csv(main_output_dir + '/' + patient + '/' 
              + measurement_date + '/' + measurement_date + '_' + patient
              +'_ECGall_' + channel + '.csv')
    
    print('Notification: Data of 5 PS settings stored!')
    print(measurement_date)

In [ ]:
# Move on to the next measurement date if data saved

if not os.path.exists(main_output_dir + '/' + patient + '/' + measurement_date  
                      + '/' + measurement_date + '_' + patient 
                      +'_ECGall_para' + '.csv'):
    print('Warning: Data not stored! Do not progress to next date!')
elif ((len(set(df_ecg_all_dict['para']['measurement'].values)) < 5) 
      |((date_idx < len(measurement_dates)
         and (measurement_date != measurement_dates[date_idx])))
         ):
    print('Warning: Data not stored! Do not progress to next date!')
else:
    if date_idx < len(measurement_dates)-1:
        print('Notification: Data appropriately stored. '
              +'You may progress to next date:')
        
        if measurement_date == measurement_dates[date_idx]:
            date_idx += 1            
            print(measurement_dates[date_idx])
    else:
        if patient == patients[patient_idx]:
            patient_idx += 1
            date_idx = 0
        print('Notification: Data appropriately stored. \n'+
              'This was the last measurement of this patient. '
              +'You may progress to next patient!')

In [ ]:
# # Save data anyway:
# df_ecg.to_csv(main_output_dir + '/' + patient + '/' + measurement_date  
#             + '/' + measurement_date + '_' + patient
#             +'_ECGs.csv')

# for _, channel in enumerate(channel_list):

#     df_ecg_all_dict[channel].to_csv(main_output_dir + '/' + patient + '/' 
#             + measurement_date + '/' + measurement_date + '_' + patient
#             +'_ECGall_' + channel + '.csv')

# date_idx += 1
# if date_idx < len(measurement_dates):
#     print('Notification: Data appropriately stored. '
#             +'You may progress to next date!')
#     print(measurement_date)
# else:
#     if patient == patients[patient_idx]:
#         patient_idx += 1
#         date_idx = 0
#     print('Notification: Data appropriately stored. \n'+
#             'This was the last measurement of this patient. '
#             +'You may progress to next patient!')


In [ ]:
# # Move on to the next measurement date
# date_idx += 1

In [ ]:
# # Move on to the previous measurement date
# date_idx -= 1

In [ ]:
# date_idx

['To next measurement date'](#section_date_step)

In [ ]:
# Move on to the next patient
# patient_idx += 1

In [ ]:
# # Move on to the previous patient
# patient_idx -= 1

['To next patient'](#section_patient_step)

In [ ]:
# gate_twice = True

# Normalisation of ETPs

In [ ]:
# # Normalise the ETPdi, PTPocc and NMCdi values relative to PEEP = 9 cmH2O:
# PEEP_norm = 9
# if len(set(df.loc[df['PEEP_set'] == PEEP_norm, 'PEEP_set'].values)) != 1:
#     print('Warning: PEEP = ', PEEP_norm, ' cmH2O is not available yet. ',
#           'Run that dataset first!')
# else:
#     # Strict
#     PTP_PEEP9_strict = np.median(output_df.loc[
#         quality_df_bool_strict.all(axis=1).values
#         & (df['PEEP_set'] == PEEP_norm), 
#         'PTP_occs']
#     )
#     ETP_PEEP9_strict = np.median(output_df.loc[
#         quality_df_bool_strict.all(axis=1).values
#         & (df['PEEP_set'] == PEEP_norm), 
#         'ETP_di_occs']
#     )
#     output_df = output_df.assign(PTP_norm_strict=
#                      output_df['PTP_occs'] / PTP_PEEP9_strict)
#     output_df = output_df.assign(ETP_norm_strict=
#                      output_df['ETP_di_occs'] / ETP_PEEP9_strict)
#     output_df = output_df.assign(NMC_di_norm_strict=
#                      output_df['PTP_norm_strict'] / 
#                      output_df['ETP_norm_strict'])
#     # Tolerant
#     PTP_PEEP9_tolerant = np.median(output_df.loc[
#         quality_df_bool_tolerant.all(axis=1).values
#         & (df['PEEP_set'] == PEEP_norm), 
#         'PTP_occs']
#     )
#     ETP_PEEP9_tolerant = np.median(output_df.loc[
#         quality_df_bool_tolerant.all(axis=1).values
#         & (df['PEEP_set'] == PEEP_norm), 
#         'ETP_di_occs']
#     )
#     output_df = output_df.assign(PTP_norm_tol=
#                      output_df['PTP_occs'] / PTP_PEEP9_tolerant)
#     output_df = output_df.assign(ETP_norm_tol=
#                      output_df['ETP_di_occs'] / ETP_PEEP9_tolerant)
#     output_df = output_df.assign(NMC_di_norm_tol=
#                      output_df['PTP_norm_tol'] / output_df['ETP_norm_tol'])

# print(output_df)

In [ ]:
# # Plot the normalised NMCdi values, permissible according to the tolerant 
# # criteria
# if len(set(df['PS_set'].values)) < 4:
#     print('Warning: Not 4 PS settings evaluated yet!')
# else:
#     passed_all_tolerant_crit = quality_df_bool_strict.all(axis=1)

#     bp = output_df[passed_all_tolerant_crit].plot.scatter('PEEP_set', 
#                                                    'NMC_di_norm_strict')
#     bp.set_ylabel('Normalised NMC_di (.)')
#     bp.set_xlabel('PEEP (cmH2O)')
#     bp.set_title('Normalised')
#     ylims = bp.set_ylim()
#     bp.set_ylim((0, ylims[1]))